In [1]:
import cv2
import redis
import numpy as np
from insightface.app import FaceAnalysis

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hostname = 'redis-18992.c258.us-east-1-4.ec2.redns.redis-cloud.com'
port = '18992'
password = 'nUoPu0pKGDk95n5zatr1N1DgTDNmnAuG'

In [3]:
r = redis.StrictRedis(host=hostname,
               port= port, password=password)

In [4]:
r.ping()

True

In [5]:
app_sc = FaceAnalysis(name='buffalo_sc',providers=['CPUExecutionProvider'])
app_sc.prepare(ctx_id=0,det_size=(640,640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\umesh/.insightface\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\umesh/.insightface\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Step 1 : Collect Person data and role

In [6]:
person_name = input("Enter your name : ")
retry = 3
for i in range(retry):
    role= input("""
    Please choose role
    1. Student
    2. Teacher
    Enter 1 for Teacher and 2 for Student
    """)
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role == 'Teacher'
            
        break   
    else:
        print("Invalid entry")
        retry -= 1
else:
    print("Maximum Attempts exceeded")

Enter your name : Umesh Khatiwada

    Please choose role
    1. Student
    2. Teacher
    Enter 1 for Teacher and 2 for Student
    2


In [7]:
name_role_dict = {'1' : 'teacher', '2': 'student'}
print(f"Name : {person_name} and Role:{role}")
person_name = '_'.join(person_name.lower().split(' '))
name_role= f'{person_name}#{name_role_dict[role]}'


Name : Umesh Khatiwada and Role:2


Step 2: Collect Samples

In [8]:

# define a video capture object 
vid = cv2.VideoCapture(0) 
face_embeddings = []
training_data_number = 0
while(True): 
      
    # Capture the video frame by frame 
    ret, frame = vid.read() 
    if ret == False:
        print("Unable to read camera")
        break
        
    results = app_sc.get(frame, max_num=1)
    
    for result in results:
        x1,y1,x2,y2 = result['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2), (0,255,0),1)
        face_data = result['embedding']
        face_embeddings.append(face_data)
        training_data_number += 1
    
    # we just want 200 training frames per person for training
    if (training_data_number == 200):
        break
        
  
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()

In [9]:
len(face_embeddings)

200

Step 3: Optimize Data and save in Redis database

In [10]:
# convert list to numpy array
face_data = np.asarray(face_embeddings)
face_data.shape

(200, 512)

In [11]:
# save only single row of  facial feature which contains 512 facial embeddings 
# obtained as the average of 200 frames we captured earlier
face_data_mean = face_data.mean(axis=0)

In [12]:
# convert the data to bytes to save to redis
face_data_byte_encoded = face_data_mean.tobytes()

In [13]:
# save data to redis and associate with the key (username#role)
# 'school:register' is the hash name , refer to earlier document for 
r.hset(name='school:register', key = name_role, value= face_data_byte_encoded)

1